<a href="https://colab.research.google.com/github/monimoreno2905/SegundoParcial/blob/main/GP_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este código contiene la implementación de un clasificador GP multiclase para m-nist. Así mismo se implementa un clasificador por autoencoder variacional para comparar las ventajas y desventajas de cada método

1. INSTALACIÓN Y DESCARGA DE LIBRERIAS

In [ ]:
!pip install gpflow --upgrade #tensorflow~=2.12.0 tensorflow-probability~=0.20.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.9/392.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.1 MB/s eta 0:00:00


In [ ]:
#Implementación de Keras dentro de tensorflow, para la definición de redes
from tensorflow import keras
#Permite especificar que un parámetro o retorno puede ser cualquier tipo de secuencia
from typing import Sequence
#Libreria para graficar
import matplotlib.pyplot as plt
#Libreria para algunas operaciones matemáticas
import numpy as np
#Libreria para utilizar lo relacionado a procesos gausianos
import gpflow
#Ignorar advertencias de tensorflow
import warnings
warnings.filterwarnings("ignore")  # ignore DeprecationWarnings from tensorflow
#Libreria para redes neuronales
import tensorflow as tf
#Libreria para disminuir hyperparametros sin afectar el rendimiento de las pruebas
from gpflow.ci_utils import reduce_in_tests
#Librerias para imprimir un resumen de los procesos gausianos y controlar si los parámetros son entrenables o no
from gpflow.utilities import print_summary, set_trainable
#Graficas
%matplotlib inline


# reproducibility:
np.random.seed(0)
tf.random.set_seed(123)

2. CARGAR LA BASE DE DATOS

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float64) / 255 #normalización de los datos
X_test = X_test.astype(np.float64) / 255
X_train, X_valid = X_train_full[:-50000], X_train_full[-50000:] #Se toman solo algunos datos, no todos
y_train, y_valid = y_train_full[:-50000], y_train_full[-50000:]

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
#Se reajusta las imágenes para que se organicen como una matriz de NxP donde N es la
#cantidad total de imágenes y P es las dimensiones (28x28=784 para m-nist)
X_train_flat = X_train.reshape(X_train.shape[0], -1)
y_train=y_train.astype(np.float64)
#Matriz de datos y etiquetas que alimenta al clasificador
data = (X_train_flat, y_train)

3. DEFINICIÓN DEL MODELO GAUSSIANO

3.1. Parametros del modelo

**Kernel matern32**

$k(x, x') = \sigma^2 \left(1 + \frac{\sqrt{3} r}{\ell}\right) \exp\left(-\frac{\sqrt{3} r}{\ell}\right)$

Donde

$sigma^2$ es la varianza del kernel.

$\ell$ es la longitud de escala.

$ r = \|x - x'\| $ es la distancia euclidiana entre los puntos \(x\) y \(x'\)

**White kernel**

$k(x, x') = \sigma^2 \delta(x, x')$

Donde

$\sigma^2$ es la varianza del ruido

 $\delta(x, x')$ es la función delta de Dirac

In [ ]:
from gpflow.kernels import SquaredExponential, ArcCosine

C=10 #Número de clases

"""Definición del Kernel
El kernel matern logra modelar funciones suaves pero con cierta flexibilidad para ajustarse a datos que pueden tener irregularidades
El kernel white introduce ruido blanco en el modelo, lo cual es útil para modelar el ruido en los datos y evitar sobreajuste
"""
#kernel = gpflow.kernels.SquaredExponential() + gpflow.kernels. ArcCosine(order=1)
kernel = gpflow.kernels.Matern32() + gpflow.kernels.White(variance=0.01)

"""Robustmax Inverse Multiclass Likelihood
La funciónlink transforma las predicciones del modelo dentro de una escala apropiada para la función probabilidad como por ejemplo sigmoide
La función invlink hace lo opuesto, es decir, convierte la escala de probabilidad en la escala original de las salidas del modelo
Lo anterior se hace debido a que likelihood no es gaussiana
"""
invlink = gpflow.likelihoods.RobustMax(C)  # Robustmax inverse link function

"""Multiclass Likelihood
Dentro de esta función, invilink se utiliza para mapear las salidas del modelo a una función de densidad acumulativa que pueda darme
los valores de probabilidad requeridos para la clasificación teniendo en cuenta que hay 10 clases.
Lo anterior se hace porque la salida no se modela directamente sino que se modela una función f con un proceso gausiano que luego
se pasa a traves de una función sigmoide o de densidad de probabilidad para obtener las probabilidades de pertenencia a cada clase.
"""
likelihood = gpflow.likelihoods.MultiClass(
    C, invlink=invlink
)

"""Variables de inducción
Se escogen algunos puntos del conjunto total de datos. Estos puntos se mantendran fijos durante el entrenamiento del modelo
y cuando ingrese un nuevo dato para predecir, se comparará las distribuciones del nuevo punto con las variables de inducción
para saber a que clase pertenece.
"""
Z = X_train_flat[::50].copy()


"""Modelo sparse variational gaussian process
Se ingresa el kernel, la probabilidad, los datos, las clases
El parametro whiten facilita la optimización de los parametros asegurando que la matriz de covarianza sea diagonal
El parametro q_diag asume una matriz de covarianza diagonal para la variación del modelo posterior disminuyendo costo computacional
"""
m = gpflow.models.SVGP(
    kernel=kernel,
    likelihood=likelihood,
    inducing_variable=Z,
    num_latent_gps=C,
    whiten=True,
    q_diag=True,
)

#Establecer que parametros no entrenar
set_trainable(m.kernel.kernels[1].variance, False) #La varianza del segundo kernel no seria ajustada durante el entrenamiento
set_trainable(m.inducing_variable, False) #Evitar que el modelo ajuste los puntos del espacio latente
print_summary(m, fmt="notebook") #Resumen del modelo

name,class,transform,prior,trainable,shape,dtype,value
SVGP.kernel.kernels[0].variance,Parameter,Softplus,,True,(),float64,1.0
SVGP.kernel.kernels[0].lengthscales,Parameter,Softplus,,True,(),float64,1.0
SVGP.kernel.kernels[1].variance,Parameter,Softplus,,False,(),float64,0.009999999999999998
SVGP.likelihood.invlink.epsilon,Parameter,Sigmoid,Beta,False,(),float64,0.0010000000000000005
SVGP.inducing_variable.Z,Parameter,Identity,,False,"(200, 784)",float64,"[[0., 0., 0...."
SVGP.q_mu,Parameter,Identity,,True,"(200, 10)",float64,"[[0., 0., 0...."
SVGP.q_sqrt,Parameter,Softplus,,True,"(200, 10)",float64,"[[1., 1., 1...."


3.2. Entrenamiento del modelo

**Ecuación Optmizador**

$
\theta_{k+1} = \theta_k - H_k^{-1} \nabla f(\theta_k)$


Donde:

$ \theta_k $ son los parámetros en la iteración k,

$ H_k^{-1}$ es una aproximación de la inversa de la matriz Hessiana de la función objetivo (f),

$\nabla f(\theta_k)$ es el gradiente de la función objetivo en $\theta_k$.

**Ecuación Función de perdida**


$\mathcal{L}(\mathbf{f}) = \mathbb{E}_{q(\mathbf{f})}[\log p(\mathbf{y} \mid \mathbf{f})] - \text{KL}(q(\mathbf{f}) \| p(\mathbf{f}))$


Donde:

 $\mathcal{L}(\mathbf{f})$  es el límite inferior variacional,

$\mathbb{E}_{q(\mathbf{f})}[\log p(\mathbf{y} \mid \mathbf{f})]$ es la expectativa de la log-verosimilitud de los datos dados las funciones latentes bajo la distribución variacional,

$\text{KL}(q(\mathbf{f}) \| p(\mathbf{f}))$ es la divergencia de Kullback-Leibler entre la distribución variacional $q(\mathbf{f})$ y la distribución a priori $p(\mathbf{f})$.



In [ ]:
"""Optimizador
Por defecto se está utilizando el optimizador L-BFGS-B (Limited-memory Broyden-Fletcher-Goldfarb-Shanno with Box constraints)
El optimizador utiliza aproximaciones a la matriz Hessiana (segunda derivada de la función de pérdida) para realizar actualizaciones de los parámetros
"""
opt = gpflow.optimizers.Scipy()

"""Se optimiza con la función de perdida Variational Lower Bound
que es el límite inferior de la log-marginal likelihood """
opt_logs = opt.minimize(
    m.training_loss_closure(data),
    m.trainable_variables, #se le indica al optimizador que ajuste los parámetros entrenables
    options=dict(maxiter=reduce_in_tests(1000)), # define el número máximo de iteraciones del optimizador
)

print_summary(m, fmt="notebook") #Se imprime resumen del modelo ya entrenado

name,class,transform,prior,trainable,shape,dtype,value
SVGP.kernel.kernels[0].variance,Parameter,Softplus,,True,(),float64,56.63459
SVGP.kernel.kernels[0].lengthscales,Parameter,Softplus,,True,(),float64,80.71305
SVGP.kernel.kernels[1].variance,Parameter,Softplus,,False,(),float64,0.009999999999999998
SVGP.likelihood.invlink.epsilon,Parameter,Sigmoid,Beta,False,(),float64,0.0010000000000000005
SVGP.inducing_variable.Z,Parameter,Identity,,False,"(200, 784)",float64,"[[0., 0., 0...."
SVGP.q_mu,Parameter,Identity,,True,"(200, 10)",float64,"[[-0.81315245, -0.67642011, -0.32684294..."
SVGP.q_sqrt,Parameter,Softplus,,True,"(200, 10)",float64,"[[0.00339651, 0.00619568, 0.0030468..."


3.3. Medidas de eficiencia

$\text{Exactitud} = \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}}$

$\text{Precisión} = \frac{\text{TP}}{\text{TP} + \text{FP}}$

$\text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}$

$\text{F1 Score} = 2 \times \frac{\text{Precisión} \times \text{Recall}}{\text{Precisión} + \text{Recall}}$

Donde

Tp son los verdaderos positivos

TN son los verdaderos negativos

Fp son los verdaderos positivos

FN son los verdaderos negativos

In [ ]:
"""Primero se hacen predicciones sobre los datos totales"""
Fmean, _ = m.predict_f(X_train_flat) #Se hace predicciones sobre el conjunto total de datos
P = m.likelihood.invlink(Fmean) #Se aplica la invilink para ajustar las probabilidades de clasificación
array = P.numpy() # Convierte el tensor a un array numpy
max_positions = np.argmax(array, axis=1) #Encuentra la clase a la que pertenece cada elemento según las probabilidades

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_true = y_train
y_pred = max_positions

#Calcular accuracy (porcentaje total de aciertos)
accuracy = np.mean(y_train == y_pred)

# Calcular precisión (mide la exactitud de las predicciones positivas de un modelo)
precision = precision_score(y_true, y_pred, average='weighted')

# Calcular recall (mide la capacidad de un modelo para identificar todos los casos positivos)
recall = recall_score(y_true, y_pred, average='weighted')

# Calcular F1 score (combina la precisión y el recall en una sola medida.)
f1 = f1_score(y_true, y_pred, average='weighted')

#Calcular matriz de confusion
cm = confusion_matrix(y_true, y_pred)

# Imprimir resultados
print(cm)
print("Precisión:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)

[[829   1  12  33   5   0  54   0   8   0]
 [  5 995   6  19   1   0   1   0   0   0]
 [ 10   2 822   8 111   2  55   0   6   0]
 [ 35   8   9 922  31   0  10   0   4   0]
 [  5   0  89  23 810   1  41   0   5   0]
 [  0   0   0   2   0 951   1  22   3  10]
 [125   4 108  33  77   0 660   0  13   1]
 [  0   0   0   0   0  34   0 944   3  41]
 [  3   2   6   5   1   1   9   3 957   3]
 [  0   0   0   0   0  11   0  26   1 962]]
Precisión: 0.8850269210698065
F1 Score: 0.8840883434615058
Accuracy: 0.8852


4. CLASIFICADOR TIPO AUTOENCODER VARIACIONAL

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Lambda, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import UpSampling2D

# 1. Preparar los datos
X_train = np.reshape(X_train, (len(X_train), 28, 28, 1))
Z=X_train[::50].copy()
Y=y_train[::50].copy()

# Convertir las etiquetas a formato categórico
y_train = to_categorical(y_train, 10)
Y= to_categorical(Y, 10)

# 2. Definir el encoder
latent_dim = 2  # Dimensionalidad del espacio latente

inputs = Input(shape=(28, 28, 1)) #Tamaño de las imágenes

# Encoder
encoder_conv1 = Conv2D(32, kernel_size=3, activation='relu', padding='same')(inputs)
encoder_pool1 = MaxPooling2D((2, 2), padding='same')(encoder_conv1)
encoder_conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(encoder_pool1)
encoder_pool2 = MaxPooling2D((2, 2), padding='same')(encoder_conv2)

# Flatten y capas densas para obtener z_mean y z_log_var
x = Flatten()(encoder_pool2)
x = Dense(512, activation='relu')(x)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# Muestreo
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Clasificación usando el espacio latente
class_output = Dense(10, activation='softmax')(z)

# Encoder Model
encoder = Model(inputs, [z_mean, z_log_var, z, class_output], name='encoder')

# Decoder Model
latent_inputs = Input(shape=(latent_dim,))
x = Dense(7 * 7 * 64, activation='relu')(latent_inputs)
x = Reshape((7, 7, 64))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)  # Upsample to 14x14
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)  # Upsample to 28x28
decoder_outputs = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Decoder Model
decoder = Model(latent_inputs, decoder_outputs, name='decoder')

# 4. Construir el VAE
vae_outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, vae_outputs, name='vae')

# 5. Definir la función de pérdida del VAE
def vae_loss(inputs, outputs, z_mean, z_log_var):
    xent_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
    kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return K.mean(xent_loss + kl_loss)

# Función de pérdida personalizada para el modelo combinado
def combined_loss(inputs, outputs):
    z_mean, z_log_var, z = encoder(inputs)
    reconstruction_loss = vae_loss(inputs, outputs, z_mean, z_log_var)
    return reconstruction_loss

# Crear el modelo combinado
combined_outputs = [vae_outputs, class_output]
combined_model = Model(inputs, combined_outputs)

# Compilar el modelo combinado
combined_model.compile(optimizer='adam',
                       loss=['binary_crossentropy', 'categorical_crossentropy'],
                       loss_weights=[0.5, 0.5])

# Entrenar el modelo
history = combined_model.fit(Z, [Z, Y], epochs=10, batch_size=64)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 789ms/step - loss: 1.5912
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 460ms/step - loss: 1.4469
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 643ms/step - loss: 1.3198
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 230ms/step - loss: 1.2726
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 363ms/step - loss: 1.1971
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 376ms/step - loss: 1.1280
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 453ms/step - loss: 1.0669
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 474ms/step - loss: 1.0223
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 629ms/step - loss: 0.9735
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 449ms/step - loss: 0.9452


In [ ]:
# Reconstrucción y clasificación
predictions = combined_model.predict(X_train)
reconstructed_images = predictions[0]
predicted_classes = np.argmax(predictions[1], axis=1)
y_test_labels = np.argmax(y_train, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 83ms/step


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_true = y_test_labels
y_pred = predicted_classes

#Calcular accuracy (porcentaje total de aciertos)
accuracy = np.mean(y_true == y_pred)

# Calcular precisión (mide la exactitud de las predicciones positivas de un modelo)
precision = precision_score(y_true, y_pred, average='weighted')

# Calcular recall (mide la capacidad de un modelo para identificar todos los casos positivos)
recall = recall_score(y_true, y_pred, average='weighted')

# Calcular F1 score (combina la precisión y el recall en una sola medida.)
f1 = f1_score(y_true, y_pred, average='weighted')

#Calcular matriz de confusion
cm = confusion_matrix(y_true, y_pred)

# Imprimir resultados
print(cm)
print("Precisión:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)

[[379   9  11 433  13   2   1  12   0  82]
 [ 29 925  13   6  52   0   0   0   0   2]
 [168 267 109  52 322   9   8  25   0  56]
 [153  44   8 477  39   0   1   2   0 295]
 [261 171  25  94 371   5   5   4   0  38]
 [  1   0  16   0   0 216  13 723   0  20]
 [297 103  71 203 150   8  13  41   0 135]
 [  0   0   0   0   0   7  32 959   0  24]
 [ 21   8  48  18  10  77  55 555   0 198]
 [  0   2   3   0   0   2  97 138   0 758]]
Precisión: 0.35990227835818295
Recall: 0.4207
F1 Score: 0.3515503799777518
Accuracy: 0.4207
